# Classifying facial expressions

In [1]:
# Module imports
import numpy as np
import pandas as pd
import seaborn as sb

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler

In [10]:
# Helpers, reading files

def load_features_and_labels(subject, expression, path = "./data/"):
    """ Reads the text files of raw data for a specified subject and
    expression.
    """
    raw_features = pd.read_table(path + subject + "_" + expression + "_datapoints.txt",
                                 sep=" ",
                                 header=0)
    
    raw_labels = pd.read_table(path + subject + "_" + expression + "_targets.txt",
                               sep=" ",
                               header=None)
    
    return raw_features, raw_labels

X_train_raw, Y_train = load_features_and_labels("a", "affirmative")

In [12]:
X_train_raw

,0.0,0x,0y,0z,1x,1y,1z,2x,2y,2z,...,96z,97x,97y,97z,98x,98y,98z,99x,99y,99z
0,1.390385e+09,244.672,208.182,895,240.949,205.708,890,235.994,205.018,883,...,923,283.559,186.549,944,291.722,189.204,0,296.547,192.722,1019
1,1.390385e+09,265.953,195.731,1016,262.556,193.301,1007,258.243,192.547,1001,...,1050,301.606,176.421,1069,308.776,180.199,0,312.501,184.498,0
2,1.390385e+09,272.755,192.786,1060,269.481,190.460,1056,265.371,189.756,1053,...,1097,307.299,174.126,1114,314.125,177.992,1140,317.529,182.324,1179
3,1.390385e+09,281.890,190.533,1129,278.861,188.023,1129,274.899,187.213,1136,...,1167,315.525,176.009,1187,322.016,180.029,0,325.041,184.442,1252
4,1.390385e+09,283.933,193.201,1159,280.955,190.877,1159,277.180,190.196,1167,...,1195,315.102,177.654,1216,321.384,181.259,1225,324.488,185.400,1284
5,1.390385e+09,284.088,193.384,1171,281.072,191.095,1171,277.301,190.434,1175,...,1204,314.990,177.709,1225,321.307,181.331,1238,324.461,185.499,1289
6,1.390385e+09,284.228,193.204,1187,281.229,191.161,1183,277.519,190.626,1191,...,1221,314.753,176.996,1238,320.950,180.441,1243,324.077,184.382,1309
7,1.390385e+09,284.239,192.134,1195,281.266,190.147,1195,277.591,189.624,1200,...,1229,314.738,176.124,1247,320.846,179.652,1256,323.896,183.603,0
8,1.390385e+09,284.200,191.798,1200,281.246,189.840,1200,277.628,189.314,1204,...,1234,314.360,175.972,1256,320.386,179.378,1256,323.465,183.242,1309
9,1.390385e+09,283.920,191.230,1200,281.046,189.334,1204,277.472,188.816,1204,...,1238,314.348,175.969,1256,320.224,179.484,1256,323.115,183.334,1314


## a)  Classify two expressions from subject A using "off the shelf" tech

NB;  Extra marks for coding my own implementation of the classifier

## b)  Use these classifiers to classify two expressions from subject B
NB;  Extra marks for using something beyond simple accuracy (recycle ROC curve code?)

## c)  Additional Analysis of classifiers - reverse roles!
Train on B, classify A, comment on difference!
Try again using a different feature representation (eg; PCA.  Can I think of something better?)

## d)  Implement a different classifier
Training on single expression, testing (on B?) and extra marks for own implementation
Training and testing on a SECOND expression
The same by inverting roles
Repeating with a different feature representation again!  
REF;  for last, try multiclass classifier on principle there should be some shared information?  Data imbalance problem

## e)  Wrap-up, compare results of the two classifiers, make comments